In [11]:
import pandas as pd
from datetime import datetime
import re
import numpy as np
pd.set_option('display.max_columns', 100)

---

In [12]:
big_startup_secsees_dataset = pd.read_csv('data/big_startup_secsees_dataset.csv')

<h2>big_startup_secsees_dataset: Startup Success/Fail Dataset from Crunchbase (2016)</h2>
Кол-во строк: 66368

DataSet for Startup Success Prediction from Crunchbase

* permalink
* name
* homepage_url
* category_list	(728 categories)
* funding_total_usd	
* status (operating, closed, acquired, ipo)
* country_code	
* state_code	
* region	
* city	
* funding_rounds	
* founded_at	
* first_funding_at	
* last_funding_at

Last date: 2016-04-02

In [13]:
big_startup_secsees_dataset.shape

(66368, 14)

In [14]:
big_startup_secsees_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   permalink          66368 non-null  object
 1   name               66367 non-null  object
 2   homepage_url       61310 non-null  object
 3   category_list      63220 non-null  object
 4   funding_total_usd  66368 non-null  object
 5   status             66368 non-null  object
 6   country_code       59410 non-null  object
 7   state_code         57821 non-null  object
 8   region             58338 non-null  object
 9   city               58340 non-null  object
 10  funding_rounds     66368 non-null  int64 
 11  founded_at         51147 non-null  object
 12  first_funding_at   66344 non-null  object
 13  last_funding_at    66368 non-null  object
dtypes: int64(1), object(13)
memory usage: 7.1+ MB


In [15]:
big_startup_secsees_dataset.sample(3)

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
36459,/organization/mobile360,Mobile360,NaN,Advertising|E-Commerce|Social Media,-,operating,MYS,14,Kuala Lumpur,Kuala Lumpur,1,NaN,2015-01-14,2015-01-14
6294,/organization/beijing-transinfo-technology-gro...,Transinfo Group,http://www.ctfo.com,Public Transportation,11623640,operating,CHN,22,Beijing,Beijing,2,2000-01-01,2010-02-01,2011-07-01
14853,/organization/dating-headshots-inc,Dating Headshots Inc.,http://www.datingheadshots.com/,Online Dating|Social Media,170000,operating,USA,CA,Los Angeles,Beverly Hills,5,2007-01-10,2007-01-05,2014-06-20


In [16]:
# Поменяем типы данных для funding_total_usd и посотрим на распределение:
big_startup_secsees_dataset.funding_total_usd = big_startup_secsees_dataset.funding_total_usd.apply(
    lambda x: None if x == '-' else x).astype('float64')

In [17]:
big_startup_secsees_dataset.funding_total_usd.describe()

count    5.358300e+04
mean     1.847860e+07
std      1.880133e+08
min      1.000000e+00
25%      3.356235e+05
50%      2.000000e+06
75%      1.000000e+07
max      3.007950e+10
Name: funding_total_usd, dtype: float64

In [18]:
# Посмотрим на распределение категорий
big_startup_secsees_dataset.category_list.str.split('|', expand=True)[0].value_counts(normalize=True)

0
Biotechnology                      0.070674
Software                           0.065549
E-Commerce                         0.043531
Advertising                        0.037915
Curated Web                        0.034752
                                     ...   
Biotechnology and Semiconductor    0.000016
Flowers                            0.000016
Registrars                         0.000016
Swimming                           0.000016
Task Management                    0.000016
Name: proportion, Length: 728, dtype: float64

In [19]:
# Посмотрим, какие статусы бывают:
big_startup_secsees_dataset.status.value_counts()

status
operating    53034
closed        6238
acquired      5549
ipo           1547
Name: count, dtype: int64

In [22]:
# Кол-во строк с неуникальным именем

len(big_startup_secsees_dataset) - big_startup_secsees_dataset.name.nunique()

266

In [23]:
# Просто исправить явные ошибки

big_startup_secsees_dataset.loc[63921, 'last_funding_at'] = '2015-12-01'
big_startup_secsees_dataset.loc[46956, 'last_funding_at'] = '2015-05-01'

---

In [24]:
acquisitions = pd.read_csv('data/archive/acquisitions.csv')
degrees = pd.read_csv('data/archive/degrees.csv')
funding_rounds = pd.read_csv('data/archive/funding_rounds.csv')
funds = pd.read_csv('data/archive/funds.csv')
investments = pd.read_csv('data/archive/investments.csv')
ipos = pd.read_csv('data/archive/ipos.csv')
milestones = pd.read_csv('data/archive/milestones.csv')
objects = pd.read_csv('data/archive/objects.csv', low_memory=False)
offices = pd.read_csv('data/archive/offices.csv')
people = pd.read_csv('data/archive/people.csv')
relationships = pd.read_csv('data/archive/relationships.csv')

<h2>Startup Investments & Startups (Crunchbase) (2014)</h2>

Venture deals, organizations, people and exits. Information about startups via Crunchbase

* acquisitions dataset
* degrees dataset
* funding_rounds dataset
* funds dataset
* investments dataset
* ipos dataset
* milestones dataset
* offices dataset
* people dataset
* relationships dataset

Last date: '2014-12-31'

* objects dataset разбили на 4 части:
       * obj_companies (196_553 entries)
       * obj_finorgs (11_652 entries)
       * obj_persons (226_708 entries)
       * obj_products (27_738 entries)

In [25]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9562 entries, 0 to 9561
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   9562 non-null   int64  
 1   acquisition_id       9562 non-null   int64  
 2   acquiring_object_id  9562 non-null   object 
 3   acquired_object_id   9561 non-null   object 
 4   term_code            1906 non-null   object 
 5   price_amount         9562 non-null   float64
 6   price_currency_code  9558 non-null   object 
 7   acquired_at          9533 non-null   object 
 8   source_url           8566 non-null   object 
 9   source_description   8590 non-null   object 
 10  created_at           9562 non-null   object 
 11  updated_at           9562 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 896.6+ KB


In [26]:
acquisitions.sample()

,id,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,source_url,source_description,created_at,updated_at
8614,8615,9497,c:84077,c:258974,NaN,640000000.0,USD,2013-08-27,http://www.bioportfolio.com/news/article/16111...,Akorn to Acquire Hi-Tech Pharmacal for $640 Mi...,2013-08-27 12:35:41,2013-08-27 12:35:41


In [27]:
degrees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109610 entries, 0 to 109609
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            109610 non-null  int64 
 1   object_id     109610 non-null  object
 2   degree_type   98389 non-null   object
 3   subject       81298 non-null   object
 4   institution   109555 non-null  object
 5   graduated_at  58054 non-null   object
 6   created_at    109610 non-null  object
 7   updated_at    109610 non-null  object
dtypes: int64(1), object(7)
memory usage: 6.7+ MB


In [28]:
degrees.sample()

,id,object_id,degree_type,subject,institution,graduated_at,created_at,updated_at
86902,86903,p:156237,BS,Information Technology,The University of Reading,NaN,2013-10-04 04:51:50,2013-10-04 04:51:50


In [29]:
funding_rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52928 entries, 0 to 52927
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        52928 non-null  int64  
 1   funding_round_id          52928 non-null  int64  
 2   object_id                 52928 non-null  object 
 3   funded_at                 52680 non-null  object 
 4   funding_round_type        52928 non-null  object 
 5   funding_round_code        52928 non-null  object 
 6   raised_amount_usd         52928 non-null  float64
 7   raised_amount             52928 non-null  float64
 8   raised_currency_code      49862 non-null  object 
 9   pre_money_valuation_usd   52928 non-null  float64
 10  pre_money_valuation       52928 non-null  float64
 11  pre_money_currency_code   26883 non-null  object 
 12  post_money_valuation_usd  52928 non-null  float64
 13  post_money_valuation      52928 non-null  float64
 14  post_m

In [30]:
funding_rounds.sample()

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at
45284,49879,49879,c:231359,2011-10-05,series-a,a,500000.0,500000.0,USD,0.0,0.0,USD,0.0,0.0,USD,0,0,1,http://www.sec.gov/Archives/edgar/data/1531353...,SEC,prasant2013,2013-09-25 06:43:09,2013-09-25 06:43:09


In [31]:
funds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1564 non-null   int64  
 1   fund_id               1564 non-null   int64  
 2   object_id             1564 non-null   object 
 3   name                  1564 non-null   object 
 4   funded_at             1449 non-null   object 
 5   raised_amount         1564 non-null   float64
 6   raised_currency_code  1564 non-null   object 
 7   source_url            1272 non-null   object 
 8   source_description    1218 non-null   object 
 9   created_at            1564 non-null   object 
 10  updated_at            1564 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 134.5+ KB


In [32]:
funds.sample()

,id,fund_id,object_id,name,funded_at,raised_amount,raised_currency_code,source_url,source_description,created_at,updated_at
1378,1504,1504,f:642,LBO Fund IV,2008-01-01,1.600000e+09,EUR,http://www.unquote.com/unquote/official-record...,Axa spinout Ardian closes fifth LBO fund on 2...,2013-10-03 11:37:39,2013-10-03 11:37:39


In [33]:
investments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80902 entries, 0 to 80901
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  80902 non-null  int64 
 1   funding_round_id    80902 non-null  int64 
 2   funded_object_id    80902 non-null  object
 3   investor_object_id  80902 non-null  object
 4   created_at          80902 non-null  object
 5   updated_at          80902 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.7+ MB


In [34]:
investments.sample()

,id,funding_round_id,funded_object_id,investor_object_id,created_at,updated_at
15460,15461,10438,c:27545,f:2619,2009-12-18 20:09:12,2009-12-18 20:09:13


In [35]:
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1259 non-null   int64  
 1   ipo_id                   1259 non-null   int64  
 2   object_id                1254 non-null   object 
 3   valuation_amount         1259 non-null   float64
 4   valuation_currency_code  1257 non-null   object 
 5   raised_amount            1259 non-null   float64
 6   raised_currency_code     699 non-null    object 
 7   public_at                659 non-null    object 
 8   stock_symbol             1259 non-null   object 
 9   source_url               191 non-null    object 
 10  source_description       180 non-null    object 
 11  created_at               1259 non-null   object 
 12  updated_at               1259 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 128.0+ KB


In [36]:
ipos.sample()

,id,ipo_id,object_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,source_url,source_description,created_at,updated_at
858,859,933,c:139880,0.0,USD,0.0,USD,2003-06-10,NYSE:KT,NaN,NaN,2013-06-03 18:50:19,2013-06-03 18:50:19


In [37]:
milestones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39456 entries, 0 to 39455
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  39456 non-null  int64 
 1   object_id           39456 non-null  object
 2   milestone_at        39456 non-null  object
 3   milestone_code      39456 non-null  object
 4   description         39456 non-null  object
 5   source_url          31401 non-null  object
 6   source_description  29268 non-null  object
 7   created_at          39456 non-null  object
 8   updated_at          39456 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.7+ MB


In [38]:
milestones.sample()

,id,object_id,milestone_at,milestone_code,description,source_url,source_description,created_at,updated_at
1071,1072,c:19562,2009-01-01,other,Version 1 of Product Launched,http://www.gocanvas.com/content/about-us/news/...,NaN,2009-03-04 20:18:25,2009-05-15 21:16:36


In [39]:
milestones.milestone_code.unique()

array(['other'], dtype=object)

In [40]:
objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462651 entries, 0 to 462650
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   462651 non-null  object 
 1   entity_type          462651 non-null  object 
 2   entity_id            462651 non-null  int64  
 3   parent_id            27715 non-null   object 
 4   name                 462647 non-null  object 
 5   normalized_name      462620 non-null  object 
 6   permalink            462651 non-null  object 
 7   category_code        123186 non-null  object 
 8   status               462651 non-null  object 
 9   founded_at           100441 non-null  object 
 10  closed_at            2809 non-null    object 
 11  domain               174942 non-null  object 
 12  homepage_url         174942 non-null  object 
 13  twitter_username     126089 non-null  object 
 14  logo_url             208850 non-null  object 
 15  logo_width       

In [41]:
objects.sample()

,id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
417220,p:79550,Person,79550,NaN,Michiel Muller,michiel muller,/person/michiel-muller,NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,2011-01-12,2011-01-12,1,1,NaN,NaN,0,0.0,NaN,NaN,0,0,robinwauters,2011-01-12 14:07:36,2011-01-12 14:07:36


In [42]:
objects.entity_type.value_counts()

entity_type
Person          226708
Company         196553
Product          27738
FinancialOrg     11652
Name: count, dtype: int64

In [43]:
offices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112718 entries, 0 to 112717
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            112718 non-null  int64  
 1   object_id     112718 non-null  object 
 2   office_id     112718 non-null  int64  
 3   description   68530 non-null   object 
 4   region        112718 non-null  object 
 5   address1      94430 non-null   object 
 6   address2      44520 non-null   object 
 7   city          107550 non-null  object 
 8   zip_code      93230 non-null   object 
 9   state_code    62017 non-null   object 
 10  country_code  112718 non-null  object 
 11  latitude      112718 non-null  float64
 12  longitude     112718 non-null  float64
 13  created_at    0 non-null       float64
 14  updated_at    0 non-null       float64
dtypes: float64(4), int64(2), object(9)
memory usage: 12.9+ MB


In [44]:
offices.sample()

,id,object_id,office_id,description,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,created_at,updated_at
67456,67457,c:43389,78685,NaN,Denver,"1100 Spruce St., Ste. 202",NaN,Boulder,80302,CO,USA,0.0,0.0,NaN,NaN


In [45]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226709 entries, 0 to 226708
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                226709 non-null  int64 
 1   object_id         226709 non-null  object
 2   first_name        226700 non-null  object
 3   last_name         226705 non-null  object
 4   birthplace        28084 non-null   object
 5   affiliation_name  226684 non-null  object
dtypes: int64(1), object(5)
memory usage: 10.4+ MB


In [46]:
people.sample()

,id,object_id,first_name,last_name,birthplace,affiliation_name
145606,145607,p:179036,Rostislav,Levkovic,NaN,Unaffiliated


In [47]:
relationships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402878 entries, 0 to 402877
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      402878 non-null  int64 
 1   relationship_id         402878 non-null  int64 
 2   person_object_id        402878 non-null  object
 3   relationship_object_id  402878 non-null  object
 4   start_at                206995 non-null  object
 5   end_at                  101046 non-null  object
 6   is_past                 402878 non-null  int64 
 7   sequence                402878 non-null  int64 
 8   title                   389526 non-null  object
 9   created_at              402878 non-null  object
 10  updated_at              402878 non-null  object
dtypes: int64(4), object(7)
memory usage: 33.8+ MB


In [48]:
relationships.sample()

,id,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at,updated_at
327560,395004,395004,p:233346,f:2257,2002-02-01,2010-09-01,1,3,Principal,2013-09-12 11:37:37,2013-09-12 11:37:37


In [49]:
objects.entity_type.unique()

array(['Company', 'FinancialOrg', 'Person', 'Product'], dtype=object)

In [50]:
obj_companies = objects[objects.entity_type == 'Company'].drop(columns=['entity_type', 'parent_id'])

In [51]:
obj_companies.shape

(196553, 38)

In [52]:
obj_companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196553 entries, 0 to 196552
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   196553 non-null  object 
 1   entity_id            196553 non-null  int64  
 2   name                 196552 non-null  object 
 3   normalized_name      196549 non-null  object 
 4   permalink            196553 non-null  object 
 5   category_code        123186 non-null  object 
 6   status               196553 non-null  object 
 7   founded_at           91227 non-null   object 
 8   closed_at            2620 non-null    object 
 9   domain               126545 non-null  object 
 10  homepage_url         126545 non-null  object 
 11  twitter_username     80591 non-null   object 
 12  logo_url             110110 non-null  object 
 13  logo_width           196553 non-null  int64  
 14  logo_height          196553 non-null  int64  
 15  short_description    7

In [53]:
obj_companies.sample(5)

,id,entity_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
28675,c:168766,168766,Powerfunder,powerfunder,/company/powerfunder,ecommerce,operating,2012-01-01,NaN,powerfunder.com,http://www.powerfunder.com,powerfunder,http://s3.amazonaws.com/crunchbase_prod_assets...,2550,1115,NaN,Crowdsourced Investment Platform,Powerfunder is a platform that enables ANYBODY...,"crowd-funding, crowdfund, crowdfunder, jobs-ac...",NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0,jorgeguatemala,2012-08-21 04:41:30,2012-08-23 18:54:30
157543,c:45881,45881,Loyaltyworks Incentive Programs,loyaltyworks incentive programs,/company/loyaltyworks,other,operating,1976-01-01,NaN,loyaltyworks.com,http://www.loyaltyworks.com,loyaltyworks,NaN,0,0,NaN,Incentive program service provider,"Loyaltyworks designs and runs online, points-b...","incentive-programs--reward-programs, loyalty-p...",USA,GA,Atlanta,Atlanta,NaN,NaN,0,0,NaN,NaN,0,0.0,2006-01-01,2006-01-01,1,3,incentive_programs,2010-04-16 14:42:27,2010-04-22 23:58:26
108307,c:252873,252873,Invnt,invnt,/company/invnt,photo_video,operating,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Invnt creates mobile applications that are com...,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0,vinthanedar,2013-08-15 08:42:41,2013-08-15 08:42:41
135701,c:279933,279933,Geoly,geoly,/company/geoly,software,operating,2013-01-01,NaN,geolyapp.com,http://www.geolyapp.com,geolyapp,http://s3.amazonaws.com/crunchbase_prod_assets...,349,349,NaN,Location-based Mobile Shopping,Geoly is a community mobile application that p...,"transaction, buy, sell, exchange, rent, mobile...",NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,3,geoly,2013-11-16 00:22:14,2013-12-09 09:24:05
131553,c:275851,275851,Amp Resources,amp resources,/company/amp-resources,NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,2,arco,2013-10-31 06:30:38,2013-10-31 06:30:38


In [54]:
obj_companies.status.value_counts()

status
operating    183441
acquired       9394
closed         2584
ipo            1134
Name: count, dtype: int64

In [55]:
obj_finorgs = objects[objects.entity_type == 'FinancialOrg'].drop(columns=[
    'entity_type', 'parent_id', 'category_code', 'status', 'first_funding_at', 
    'last_funding_at', 'funding_rounds', 'funding_total_usd'
])

In [56]:
obj_finorgs.shape

(11652, 32)

In [57]:
obj_finorgs.sample()

,id,entity_id,name,normalized_name,permalink,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
207482,f:9064,9064,Mitsui Sumitomo Insurance Venture Capital,mitsui sumitomo insurance venture,/financial-organization/mitsui-sumitomo-insura...,2012-10-01,NaN,msivc.co.jp,http://www.msivc.co.jp/funds_en.html,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,106,46,NaN,NaN,NaN,NaN,JPN,NaN,Tokyo,Tokyo,2012-02-21,2012-02-21,1,1,NaN,NaN,0,0,NaN,2012-04-25 08:34:18,2013-06-06 16:53:35


In [58]:
obj_persons = objects[objects.entity_type == 'Person'].drop(columns=[
    'entity_type', 'category_code', 'status', 'founded_at',
    'closed_at', 'short_description', 'description', 'country_code',
    'state_code', 'city', 'region', 'first_funding_at', 'last_funding_at',
    'funding_rounds', 'funding_total_usd'
])

In [59]:
obj_persons.shape

(226708, 25)

In [60]:
obj_persons.sample()

,id,entity_id,parent_id,name,normalized_name,permalink,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,overview,tag_list,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
395741,p:57864,57864,NaN,Miroslav Solanka,miroslav solanka,/person/miroslav-solanka,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,1,miro,2010-05-03 11:15:43,2010-05-03 21:03:10


In [61]:
obj_products = objects[objects.entity_type == 'Product'].drop(columns=[
    'entity_type', 'category_code', 'short_description', 'description',
    'country_code', 'state_code', 'city', 'region', 'first_investment_at',
    'last_investment_at', 'investment_rounds', 'invested_companies',
    'first_funding_at', 'last_funding_at', 'funding_rounds',
    'funding_total_usd', 'relationships', 'created_by'
])

In [62]:
obj_products.shape

(27738, 22)

In [63]:
obj_products.sample()

,id,entity_id,parent_id,name,normalized_name,permalink,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,overview,tag_list,first_milestone_at,last_milestone_at,milestones,created_at,updated_at
436762,r:12535,12535,c:17932,Nigglebusiness,nigglebusiness,/product/nigglebusiness,operating,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,2009-01-28 17:05:48,2009-01-30 23:47:52


---

<h2>Наченем слияния</h2>
Мы нашли 2 подходящие базы, чтобы начать формитровать нашу бд

In [64]:
"""
Сравним 2 очень похожих датасета с кранчбейза:
big_startup_secsees_dataset имеет более новые и полные данные.
Т.е. сначала надо обновить большой кранчбейс п.4 obj_companies 2013г./196_553 строк
маленьким п.3 big_startup_secsees_dataset 2015г./66_368 строк
далее добавляем п.2 startups (слоганы и описание) 2021г?/42_038 строк
и п.1 best_funded (фичи внешнего влияния) 2005г?/60 строк

"""
obj_companies[obj_companies.name == 'Solidia Technologies'][['permalink', 'name', 'homepage_url',
       'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'first_funding_at',
       'last_funding_at']]

,permalink,name,homepage_url,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
12642,/company/solidia-technologies,Solidia Technologies,http://www.solidiatech.com,27000000.0,operating,USA,NJ,piscataway,Piscataway,1,NaN,2012-02-01,2012-02-01


In [65]:
big_startup_secsees_dataset[big_startup_secsees_dataset.name == 'Solidia Technologies']

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
53206,/organization/solidia-technologies,Solidia Technologies,http://www.solidiatech.com,Clean Technology,27000000.0,operating,USA,NJ,Newark,Piscataway,2,2008-01-01,2012-02-01,2014-12-09


In [66]:
# Удалим ненужные признаки: дата создания записи и имя сотрудника crunchbase

obj_companies.drop(columns=['created_by', 'created_at'], inplace=True)

In [67]:
"""
Обновим данные obj_companies с помощью big_startup_secsees_dataset, затем сольем их, чтобы добавить новые данные.
Для сливания используем новое поле merge на основе последней части permalink, как самое уникальное
"""

'\nОбновим данные obj_companies с помощью big_startup_secsees_dataset, затем сольем их, чтобы добавить новые данные.\nДля сливания используем новое поле merge на основе последней части permalink, как самое уникальное\n'

In [68]:
obj_companies['linkcut'] = obj_companies.permalink.str.split('/', expand=True)[2]

In [69]:
# Проверим уникальность полученных кусков ссылок. К сожалению, 41 значение неуникально.

len(obj_companies['linkcut']) - obj_companies['linkcut'].nunique()

41

In [70]:
# На примере видно, что у идентичных объектов будут ссылки на разные relations
# т.е. хотя объект явно задвоен, чтобы ликвидировать эту проблему, придется исправилять все связанные таблицы
# В автоматическом режиме такое сделать не получится, поэтому просто сотрем дубли, 
# вряд ли это приведет к потери значимых данных

obj_companies[obj_companies['linkcut'] == 'bboescape']

,id,entity_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,updated_at,linkcut
62326,c:205898,205898,BBOescape,bboescape,/company/bboescape,games_video,operating,2013-06-01,NaN,bboescape.com,http://www.BBOescape.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,146,30,NaN,Escape Stress and Get Rewards,BBOescape.com provides a one stop shop for new...,"destress, meme, gossip, games, workout, health...",CAN,NaN,Ottawa,Ottawa,NaN,NaN,0,0,NaN,NaN,0,0.0,2013-03-30,2013-03-30,1,2,2013-10-23 12:14:16,bboescape
62327,c:205899,205899,BBOescape,bboescape,/company/bboescape,games_video,operating,2013-03-30,NaN,bboescape.com,http://www.BBOescape.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,146,30,NaN,Escape Stress and Get Rewards,BBOescape.com provides a one stop shop for new...,"destress, meme, gossip, games, workout, health...",CAN,NaN,Ottawa,Ottawa,NaN,NaN,0,0,NaN,NaN,0,0.0,2013-03-30,2013-03-30,1,2,2013-05-08 04:36:29,bboescape
62330,c:205900,205900,BBOescape,bboescape,/company/bboescape,games_video,operating,2013-03-30,NaN,bboescape.com,http://www.BBOescape.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,146,30,NaN,Escape Stress and Get Rewards,BBOescape.com provides a one stop shop for new...,"destress, meme, gossip, games, workout, health...",CAN,NaN,Ottawa,Ottawa,NaN,NaN,0,0,NaN,NaN,0,0.0,2013-03-30,2013-03-30,1,2,2013-05-08 04:36:18,bboescape


In [71]:
# Соберем индексы дубликатов

list_of_doubles = list(obj_companies['linkcut'].value_counts().head(40).index)
list_of_doubled_index = []
for double in list_of_doubles:
    list_of_doubled_index.extend((obj_companies[obj_companies['linkcut'] == double].index)[1:])
print(list_of_doubled_index)

[62327, 62330, 84825, 94141, 121669, 120596, 124492, 102648, 84895, 84896, 84897, 84898, 88800, 173437, 32924, 94143, 101014, 91652, 137533, 82244, 116220, 116219, 130428, 82243, 111265, 87664, 87663, 129202, 87662, 101013, 80810, 84824, 83659, 89864, 127865, 81183, 81184, 101012, 84821, 84817, 84823]


In [72]:
# Удалим их

obj_companies.drop(index=list_of_doubled_index, inplace=True)

In [73]:
# Проверим, все ли дубли удалились

len(obj_companies['linkcut']) - obj_companies['linkcut'].nunique()

0

In [74]:
# Проверим второй датасет на наличие дублей 

big_startup_secsees_dataset['linkcut'] = big_startup_secsees_dataset.permalink.str.split('/', expand=True)[2]

In [75]:
len(big_startup_secsees_dataset['linkcut']) - big_startup_secsees_dataset['linkcut'].nunique()

0

In [76]:
# Проверим даты на аномалии и исключим странные (последнее обновленее базы - '2014-12-31'):
    
for column in ['founded_at', 'closed_at', 'first_investment_at', 'last_investment_at', 
               'first_funding_at', 'last_funding_at', 'first_milestone_at', 'last_milestone_at', 'updated_at']:
    obj_companies[column] = pd.to_datetime(obj_companies[column], errors='coerce')
    
# По факту последняя адекватная дата - 2014-12-31

In [78]:
# Проверим даты на аномалии и исключим странные (последнее обновленее базы - 2015-12-12):
    
for column in ['founded_at', 'first_funding_at', 'last_funding_at']:
    big_startup_secsees_dataset[column] = pd.to_datetime(big_startup_secsees_dataset[column], errors='coerce')
    
# По факту последняя адекватная дата - 2016-04-02

In [79]:
# Проверим даты на аномалии и исправим странные (последняя адекватная дата - 2016-04-02)
big_startup_secsees_dataset.loc[big_startup_secsees_dataset.founded_at > pd.to_datetime('2016-04-02'), ['founded_at']]

,founded_at
16681,2105-02-17
32736,2041-09-21
53396,2104-01-01


In [80]:
# Исправим аномалии
big_startup_secsees_dataset.loc[16681, 'founded_at'] = pd.to_datetime('2015-02-17')
big_startup_secsees_dataset.loc[32736, 'founded_at'] = pd.to_datetime('2014-09-21')
big_startup_secsees_dataset.loc[53396, 'founded_at'] = pd.to_datetime('2014-01-01')

In [81]:
# Обновим данные следующим образом:
# Перезапишем поля: 'name', 'permalink', 'homepage_url', 'status', 'country_code', 'state_code', 'region', 'city'
# Заполним если нулевое значение максимальным значением

list_of_linkcuts = list(obj_companies['linkcut'])
for big_index in range(66368):
    big_linkcut = big_startup_secsees_dataset.loc[big_index, ['linkcut']][0]
    if big_linkcut in list_of_linkcuts:
        obj_index = obj_companies[obj_companies['linkcut'] == big_linkcut].index[0]
        obj_companies.loc[obj_index, ['name', 'permalink', 'homepage_url',
           'status', 'country_code', 'state_code', 'region',
           'city']] = big_startup_secsees_dataset.loc[big_index, ['name', 'permalink', 'homepage_url',
           'status', 'country_code', 'state_code', 'region',
           'city']]
        obj_companies.loc[obj_index, ['funding_total_usd']] = max(obj_companies.loc[obj_index, ['funding_total_usd']][0], 
                                     big_startup_secsees_dataset.loc[big_index, ['funding_total_usd']][0])
        
        obj_companies.loc[obj_index, ['funding_rounds']] = max(obj_companies.loc[obj_index, ['funding_rounds']][0], 
                                     big_startup_secsees_dataset.loc[big_index, ['funding_rounds']][0])
        
        if obj_companies.loc[obj_index, ['founded_at']][0] < pd.to_datetime('2100'):
            if big_startup_secsees_dataset.loc[big_index, ['founded_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['founded_at']] = max(obj_companies.loc[obj_index, ['founded_at']][0],
                                                                   big_startup_secsees_dataset.loc[big_index, ['founded_at']][0])
        else:
            if big_startup_secsees_dataset.loc[big_index, ['founded_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['founded_at']] = big_startup_secsees_dataset.loc[big_index, ['founded_at']][0]
        
        if obj_companies.loc[obj_index, ['first_funding_at']][0] < pd.to_datetime('2100'):
            if big_startup_secsees_dataset.loc[big_index, ['first_funding_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['first_funding_at']] = max(obj_companies.loc[obj_index, ['first_funding_at']][0],
                                                                   big_startup_secsees_dataset.loc[big_index, ['first_funding_at']][0])
        else:
            if big_startup_secsees_dataset.loc[big_index, ['first_funding_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['first_funding_at']] = big_startup_secsees_dataset.loc[big_index, ['first_funding_at']][0]
        
        if obj_companies.loc[obj_index, ['last_funding_at']][0] < pd.to_datetime('2100'):
            if big_startup_secsees_dataset.loc[big_index, ['last_funding_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['last_funding_at']] = max(obj_companies.loc[obj_index, ['last_funding_at']][0],
                                                                   big_startup_secsees_dataset.loc[big_index, ['last_funding_at']][0])
        else:
            if big_startup_secsees_dataset.loc[big_index, ['last_funding_at']][0] < pd.to_datetime('2100'):
                obj_companies.loc[obj_index, ['last_funding_at']] = big_startup_secsees_dataset.loc[big_index, ['last_funding_at']][0]                 
        
        print(big_linkcut, '+')
    else:
        print(big_linkcut, '-')
        continue

-fame -
-qounter -
-the-one-of-them-inc- -
0-6-com +
004-technologies -
01games-technology -
0ndine-biomedical-inc -
0xdata +
1 +
1-2-3-listo -
1-4-all -
1-618-technology -
1-800-dentist +
1-800-doctors +
1-800-publicrelations-inc- -
1-mainstream -
1-of-99 -
10-20-media +
10-minutes-with -
1000-corks -
1000-markets +
1000chi -
1000lookz -
1000memories +
1000museums-com +
1001-menus +
1001pharmacies-com +
1006-tv -
100du-tv +
100e-com +
100health -
100plus +
1010data +
10alike -
10bestthings +
10k-2 -
10sec -
10seconds-software +
10six -
10tracks-2 -
10x-genomics -
10x-technologies +
10x10-room +
10°north -
11-health -
115-network-disks -
117go -
11i-solutions +
12-labs -
12-star-survival -
120-sports -
121-rentals -
121cast +
121nexus +
1234enter +
123contactform +
123feng-com -
123greetings -
123people +
1248 -
128-technology -
12bis -
12cm -
12return -
12society +
1366-technologies +
139shop -
13th-lab +
140-proof +
140fire +
1417 -
15fen -
15five +
15minutesnow +
15shopstop -
16-mil

In [82]:
obj_companies_2015 = pd.merge(obj_companies, big_startup_secsees_dataset, how='outer')

In [83]:
# Заполним normalized_name. В дальнейшем будем использовать его для поиска дубликатов
# Также надо проверить по доменам

obj_companies_2015['normalized_name'] = obj_companies_2015.name.str.lower()

In [84]:
# Заполним category_code, если пустой, первым значением из category_list

obj_companies_2015.category_code.fillna(obj_companies_2015.category_list.str.split('|', expand=True)[0], inplace=True)

In [85]:
# Посмотрим, в чем разница между полями: 

obj_companies_2015[obj_companies_2015.short_description.notna() & obj_companies_2015.description.notna() & obj_companies_2015.overview.notna()][['short_description','description','overview']].sample()

,short_description,description,overview
178854,Touchstorm is an online video content marketin...,Video Marketing Innovators,Touchstorm is a online video distribution comp...


In [86]:
"""
Опишем признаки:

id                     'type of object': id 
entity_id              id
name                   name of a company
normalized_name        name.lower()
permalink              Crunchbase URL
category_code          category
status                 current status (12/2015)
founded_at             Date the entity was founded at
closed_at              Date the entity was closed at
domain                 URL Domain 
homepage_url           URL of entity's official site
twitter_username       official Twitter account
logo_url               Logo URL
logo_width              
logo_height             
short_description      long description 
description            short description (the fields are mixed). tag_list might be extracted from it
overview               long description. but might differ from short_description field
tag_list                
country_code            
state_code              
city                    
region                  
first_investment_at    Date of the first made investment
last_investment_at     Date of the latest made investment
investment_rounds      Number of investment rounds participated in
invested_companies     Number of companies invested in
first_funding_at       Date of the first funding round
last_funding_at        Date of the latest funding round
funding_rounds         Number of funding rounds
funding_total_usd      Total funds raised in USD
first_milestone_at     Date of the first milestone
last_milestone_at      Date of the latest milestone
milestones             Number of milestones the entity has
relationships          Number of relationships the entity has
updated_at             Date the record was updated at (if nan - filled with the last event fro the entity)
category_list          

"""

"\nОпишем признаки:\n\nid                     'type of object': id \nentity_id              id\nname                   name of a company\nnormalized_name        name.lower()\npermalink              Crunchbase URL\ncategory_code          category\nstatus                 current status (12/2015)\nfounded_at             Date the entity was founded at\nclosed_at              Date the entity was closed at\ndomain                 URL Domain \nhomepage_url           URL of entity's official site\ntwitter_username       official Twitter account\nlogo_url               Logo URL\nlogo_width              \nlogo_height             \nshort_description      long description \ndescription            short description (the fields are mixed). tag_list might be extracted from it\noverview               long description. but might differ from short_description field\ntag_list                \ncountry_code            \nstate_code              \ncity                    \nregion                  \nfirst_inv

In [87]:
# Заменим permalink на полноценную ссылку crunchbase_profile

obj_companies_2015['crunchbase_profile'] = obj_companies_2015.permalink.apply(lambda x: 'http://www.crunchbase.com' + x)
obj_companies_2015.drop(columns=['permalink'], inplace=True)

In [97]:
# obj_companies_2015.to_pickle('obj_companies_2016.pkl')
# obj_companies_2015 = pd.read_pickle('obj_companies_2016.pkl')

In [89]:
# Заполним пустые updated_at последним учтенным в базе событием

obj_companies_2015['updated_at'].fillna(obj_companies_2015[[
    'founded_at', 'closed_at', 'last_investment_at', 'last_funding_at', 
    'last_milestone_at']].max(axis=1), inplace=True)

In [90]:
# Сконвертируем строку category_list в список 

obj_companies_2015.category_list = obj_companies_2015.category_list.str.split('|')

In [91]:
# obj_companies_2015 = pd.read_pickle('obj_companies_2015_0409')

In [93]:
# Подгрузим датасет городов с кэгла

worldcities = pd.read_csv('data/worldcities.csv')
worldcities.shape

(42905, 11)

In [94]:
worldcities.sample()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
10315,Samālūţ,Samalut,28.3122,30.71,Egypt,EG,EGY,Al Minyā,NaN,91475.0,1818180512


In [95]:
# Добавим поле country вместо country_code

county_codes = worldcities.groupby(['iso3', 'country']).population.sum().reset_index()[['iso3', 'country']]
obj_companies_2015 = obj_companies_2015.merge(county_codes, how='left', left_on='country_code', right_on='iso3')
obj_companies_2015.drop(columns=['iso3', 'country_code'], inplace=True)

In [96]:
# Заменим значения status

obj_companies_2015.status = obj_companies_2015.status.apply(lambda x: 1 if x == 'operating'
                                                            else 2 if x == 'acquired'
                                                            else 3 if x == 'ipo'
                                                            else 4 if x == 'closed'
                                                            else 0)

In [98]:
# Зададим бд для статусов компаний

status_bd = pd.DataFrame({'status_id': [0, 1, 2, 3, 4], 'status': ['unknown', 'operating', 'acquired', 'ipo', 'closed']})
status_bd

,status_id,status
0,0,unknown
1,1,operating
2,2,acquired
3,3,ipo
4,4,closed


In [99]:
# Посмотрим, что получилось
#
# Увеличили кол-во строк с 196_549 до 234_973
# Значение status (может использоваться как контрольное для расчета выживаемости) 100%
# category_code увеличили с 123_186 до 161_209 строк
# first_funding увеличили с 31_507 до 76_858
# last_funding_at с 31_507 до 76_885
# funding_rounds с 196_553 до 234_974
# funding_total_usd с 196_553 до 222_189

obj_companies_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234974 entries, 0 to 234973
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   196512 non-null  object        
 1   entity_id            196512 non-null  float64       
 2   name                 234972 non-null  object        
 3   normalized_name      234972 non-null  object        
 4   category_code        161209 non-null  object        
 5   status               234974 non-null  int64         
 6   founded_at           122739 non-null  datetime64[ns]
 7   closed_at            2620 non-null    datetime64[ns]
 8   domain               126537 non-null  object        
 9   homepage_url         163110 non-null  object        
 10  twitter_username     80587 non-null   object        
 11  logo_url             110103 non-null  object        
 12  logo_width           196512 non-null  float64       
 13  logo_height   

In [100]:
# obj_companies_2015.to_pickle('obj_companies_2016.pkl')
# obj_companies_2015 =  pd.read_pickle('obj_companies_2015_0409.pkl')

---

### Y-Combinator/Startups.csv

In [135]:
y_startups = pd.read_csv('https://raw.githubusercontent.com/ali-ce/datasets/master/Y-Combinator/Startups.csv')
y_startups.shape

(688, 19)

In [136]:
y_startups.sample()

,Company,Satus,Year Founded,Mapping Location,Description,Categories,Founders,Y Combinator Year,Y Combinator Session,Investors,Amounts raised in different funding rounds,Office Address,Headquarters (City),Headquarters (US State),Headquarters (Country),Logo,Seed-DB / Mattermark Profile,Crunchbase / Angel List Profile,Website
259,Bountii,Operating,NaN,Palo Alto - California - USA,Bountii is a price search engine focusing on e...,"Shopping, Search","John Puskarich, Samir Meghani",2007,Summer,Y Combinator,"$15,000","Palo Alto, CA, USA",Palo Alto,California,USA,http://a1.images.crunchbase.com/image/upload/c...,http://www.seed-db.com/companies/view?companyi...,http://www.crunchbase.com/organization/bountii,http://bountii.com


In [137]:
y_startups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 19 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Company                                     688 non-null    object 
 1   Satus                                       688 non-null    object 
 2   Year Founded                                151 non-null    float64
 3   Mapping Location                            488 non-null    object 
 4   Description                                 605 non-null    object 
 5   Categories                                  547 non-null    object 
 6   Founders                                    449 non-null    object 
 7   Y Combinator Year                           688 non-null    int64  
 8   Y Combinator Session                        688 non-null    object 
 9   Investors                                   688 non-null    object 
 10  Amounts raised

In [138]:
# Заменим сессию комбинатора на дату

y_startups['Y Combinator Session'] = y_startups['Y Combinator Session'].apply(lambda x: '02-01' if x == 'Winter'
                                        else '08-01')
y_startups['y_combinator_year'] = pd.to_datetime(y_startups['Y Combinator Year'].astype('str') 
                                                 + '-' + y_startups['Y Combinator Session'])

In [139]:
# Удалим лишние столбцы

y_startups.drop(columns=['Mapping Location', 'Y Combinator Year', 'Y Combinator Session'], inplace=True)

In [140]:
# Переименуем столбцы

y_startups = y_startups.rename(columns={'Company': 'name',
                   'Satus': 'status',
                   'Year Founded': 'founded_at', 
                   'Description': 'description',
                   'Categories': 'category_list',
                   'Founders': 'founders',
                   'Investors': 'investors',
                   'Amounts raised in different funding rounds': 'funding_total_usd',
                   'Office Address': 'address',
                   'Headquarters (City)': 'city', 
                   'Headquarters (US State)': 'state',
                   'Headquarters (Country)': 'country', 
                   'Logo': 'logo_url',
                   'Seed-DB / Mattermark Profile': 'seed_profile',
                   'Crunchbase / Angel List Profile': 'crunchbase_profile', 
                   'Website': 'homepage_url'})

In [141]:
# Поменяем формат даты

y_startups.founded_at = pd.to_datetime(y_startups.founded_at, format='%Y')

In [142]:
# Создадим столбец кол-ва раундов инвестиций

y_startups['funding_rounds'] = y_startups.funding_total_usd.str.split(', ').apply(
    lambda x: len(x) if type(x) != float else 0)

In [143]:
# Посчитаем общую сумму привелеченных инвестиций. Если сумма не раскрыта - присваивается '1'

funds = y_startups.funding_total_usd.str.split(
    ', ', expand=True).replace(
    r',','',regex=True).replace(
    r'undisclosed amount','1',regex=True)

for i in [0,2]:
    funds0 = funds[i].str.strip("$").fillna(0).str.split('.', expand=True).fillna("")
    funds0['total'] = funds0[0] + funds0[1]
    funds[i] = funds0['total']
    funds[i] = funds[i].apply(lambda x: int(x) if str(x).isdigit() else 0)
for i in [1,3,4,5,6,7]:
    funds[i] = funds[i].str.strip("$").fillna(0).apply(lambda x: int(x) if str(x).isdigit() else 0)

funds['total'] = funds[0] + funds[1] + funds[2] + funds[3] + funds[4] + funds[5] + funds[6] + funds[7]

y_startups.funding_total_usd = funds['total']

In [144]:
# Создадим поле normalized_name

y_startups['normalized_name'] = y_startups.name.str.lower()

In [145]:
# Заменим значения status

y_startups.status = y_startups.status.apply(lambda x: 1 if x == 'Operating'
                                                            else 4 if x == 'Dead'
                                                            else 2 if x == 'Exited'
                                                            else 0)

In [146]:
# Добавим дату обновления 
y_startups['updated_at'] = y_startups.y_combinator_year

In [147]:
set(y_startups.country.unique()) - set(obj_companies_2015.country.unique())

{'Czech Republic', 'USA'}

In [148]:
# Унифицируем признак country

y_startups.country = y_startups.country.apply(lambda x: 'United States' if x == 'USA'
                        else 'Czechia' if x == 'Czech Republic'
                        else x)

In [149]:
# Добавим поле category_code

y_startups['category_code'] = y_startups.category_list.str.split(',').apply(lambda x: x[0] if type(x) != float else x)

In [150]:
# Создадим поле для точного мерджа 

y_startups['linkcut'] = y_startups.crunchbase_profile.str.split('/', expand=True)[4]

In [151]:
# Объединим базы данных obj_companies_2015 и y_startups
y_obj_companies_2015 = obj_companies_2015.merge(y_startups, on='linkcut', how='outer')

In [152]:
# Аккуратно обновим данные
y_obj_companies_2015.name_x.fillna(y_obj_companies_2015.name_y, inplace=True)
y_obj_companies_2015.normalized_name_x.fillna(y_obj_companies_2015.normalized_name_y, inplace=True)
y_obj_companies_2015.category_code_x.fillna(y_obj_companies_2015.category_code_y, inplace=True)

y_obj_companies_2015.status_x = y_obj_companies_2015[['status_x', 'status_y']].max(axis=1)

y_obj_companies_2015.founded_at_x.fillna(y_obj_companies_2015.founded_at_y, inplace=True)
y_obj_companies_2015.homepage_url_x.fillna(y_obj_companies_2015.homepage_url_y, inplace=True)
y_obj_companies_2015.logo_url_x.fillna(y_obj_companies_2015.logo_url_x, inplace=True)
y_obj_companies_2015.overview.fillna(y_obj_companies_2015.description_y, inplace=True)
y_obj_companies_2015.overview.fillna(y_obj_companies_2015.short_description, inplace=True) # Delete this column, it's almost empty and greatly duplicates overview

y_obj_companies_2015.category_list_y = y_obj_companies_2015.category_list_y.apply(lambda y: ",".join(y) if isinstance(y, list) else y)
y_obj_companies_2015.category_list_x = y_obj_companies_2015.category_list_x.apply(lambda x: ",".join(x) if isinstance(x, list) else x)
y_obj_companies_2015.category_list_x.fillna(y_obj_companies_2015.category_list_y, inplace=True)

y_obj_companies_2015.funding_total_usd_x = y_obj_companies_2015[['funding_total_usd_x', 'funding_total_usd_y']].max(axis=1)

y_obj_companies_2015.city_x.fillna(y_obj_companies_2015.city_y, inplace=True)
y_obj_companies_2015.country_x.fillna(y_obj_companies_2015.country_y, inplace=True)
y_obj_companies_2015.crunchbase_profile_x.fillna(y_obj_companies_2015.crunchbase_profile_y, inplace=True)

y_obj_companies_2015.funding_rounds_x = y_obj_companies_2015[['funding_rounds_x', 'funding_rounds_y']].max(axis=1)

y_obj_companies_2015.updated_at_x = y_obj_companies_2015[['updated_at_x', 'updated_at_y']].max(axis=1)

y_obj_companies_2015.drop(columns=['name_y',
       'status_y', 'founded_at_y', 'short_description', 'description_y', 'category_list_y',
       'funding_total_usd_y', 'city_y',
       'country_y', 'logo_url_y',
       'crunchbase_profile_y', 'homepage_url_y',
       'funding_rounds_y', 'normalized_name_y', 'category_code_y', 'updated_at_y'], inplace=True)

In [153]:
y_obj_companies_2015.rename(columns={'name_x': 'name', 
                                     'normalized_name_x': 'normalized_name', 
                                     'category_code_x': 'category_code',
                                     'status_x': 'status', 
                                     'founded_at_x': 'founded_at', 
                                     'homepage_url_x': 'homepage_url',
                                     'logo_url_x': 'logo_url', 
                                     'description_x': 'description', 
                                     'city_x': 'city', 
                                     'funding_rounds_x': 'funding_rounds',
                                     'funding_total_usd_x': 'funding_total_usd', 
                                     'updated_at_x': 'updated_at',
                                     'category_list_x': 'category_list', 
                                     'crunchbase_profile_x': 'crunchbase_profile', 
                                     'country_x': 'country'}, inplace=True)

In [154]:
# Проверим, как перенеслись данные
names = y_obj_companies_2015.normalized_name
y_obj_companies_2015[names.isin(names[names.duplicated()])].sort_values("normalized_name")\
.sort_values(by=["linkcut", 'id', 'normalized_name']).head(8)

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,description,overview,tag_list,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,updated_at,linkcut,category_list,crunchbase_profile,country,founders,investors,address,state,seed_profile,y_combinator_year
173765,c:54748,54748.0,1-800-DENTIST,1-800-dentist,health,1.0,1986-01-01,NaT,1800dentist.com,http://www.1800dentist.com,1800DENTIST,http://s3.amazonaws.com/crunchbase_prod_assets...,275.0,55.0,NaN,1-800-DENTIST® is only the best way to find a ...,NaN,CA,Los Angeles,Los Angeles,NaT,NaT,0.0,0.0,2010-08-19,2010-08-19,1.0,0.000000e+00,2010-01-01,2010-08-19,2.0,2.0,2013-03-30 17:08:54,1-800-dentist,NaN,http://www.crunchbase.com/organization/1-800-d...,United States,NaN,NaN,NaN,NaN,NaN,NaT
173766,NaN,NaN,1-800-DENTIST,1-800-dentist,Health and Wellness,1.0,1986-01-01,NaT,NaN,http://www.1800dentist.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,Los Angeles,Los Angeles,NaT,NaT,NaN,NaN,2010-08-19,2010-08-19,1.0,NaN,NaT,NaT,NaN,NaN,2010-08-19 00:00:00,1-800-dentist,Health and Wellness,http://www.crunchbase.com/organization/1-800-d...,United States,NaN,NaN,NaN,NaN,NaN,NaT
163560,c:44927,44927.0,10-20 Media,10-20 media,ecommerce,1.0,2001-01-01,NaT,10-20media.com,http://www.10-20media.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,272.0,91.0,Home and Garden Technology,"10-20 Media, Inc. operates online marketplaces...",NaN,MD,Woodbine,Baltimore,NaT,NaT,0.0,0.0,2010-03-30,2011-12-28,4.0,2.050000e+06,NaT,NaT,0.0,0.0,2010-04-06 12:04:54,10-20-media,NaN,http://www.crunchbase.com/organization/10-20-m...,United States,NaN,NaN,NaN,NaN,NaN,NaT
163561,NaN,NaN,10-20 Media,10-20 media,E-Commerce,1.0,2001-01-01,NaT,NaN,http://www.10-20media.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MD,Woodbine,Baltimore,NaT,NaT,NaN,NaN,2009-06-18,2011-12-28,4.0,2.050000e+06,NaT,NaT,NaN,NaN,2011-12-28 00:00:00,10-20-media,E-Commerce,http://www.crunchbase.com/organization/10-20-m...,United States,NaN,NaN,NaN,NaN,NaN,NaT
52268,c:191780,191780.0,1000museums.com,1000museums.com,web,1.0,2008-01-01,NaT,1000museums.com,http://www.1000museums.com,1000Museums,http://s3.amazonaws.com/crunchbase_prod_assets...,277.0,40.0,NaN,1000Museums (1KM) is a global network of museu...,NaN,MA,Lenox,MA - Other,NaT,NaT,0.0,0.0,2011-04-05,2015-11-13,9.0,6.795451e+06,2013-02-16,2013-02-16,1.0,1.0,2013-12-03 11:10:42,1000museums-com,NaN,http://www.crunchbase.com/organization/1000mus...,United States,NaN,NaN,NaN,NaN,NaN,NaT
52269,NaN,NaN,1000museums.com,1000museums.com,Curated Web,1.0,2008-01-01,NaT,NaN,http://www.1000museums.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MA,Lenox,MA - Other,NaT,NaT,NaN,NaN,2009-10-14,2015-11-13,9.0,6.795451e+06,NaT,NaT,NaN,NaN,2015-11-13 00:00:00,1000museums-com,Curated Web,http://www.crunchbase.com/organization/1000mus...,United States,NaN,NaN,NaN,NaN,NaN,NaT
39066,c:177864,177864.0,ZenChef,zenchef,web,1.0,2010-11-20,NaT,1001menus.com,http://zenchef.com/en/,1001menus,http://s3.amazonaws.com/crunchbase_prod_assets...,1024.0,768.0,Local businesses online marketing,1001 Menus is an easy solution for small busin...,"local-businesses-marketing, online-menu, mobil...",A8,Paris,Paris,NaT,NaT,0.0,0.0,2013-11-13,2015-11-04,5.0,1.066177e+07,2013-01-01,2013-01-01,1.0,5.0,2012-11-22 08:01:21,1001-menus,NaN,http://www.crunchbase.com/organization/1001-menus,France,NaN,NaN,NaN,NaN,NaN,NaT
39067,NaN,NaN,ZenChef,zenchef,Local Businesses,1.0,2010-11-20,NaT,NaN,http://zenchef.com/en/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,Paris,Paris,NaT,NaT,NaN,NaN,2012-12-15,2015-11-04,5.0,1.066177e+07,NaT,NaT,NaN,NaN,2015-11-04 00:00:00,1001-menus,"Local Businesses,Restaurants",http://www.crunchbase.com/organization/1001-menus,France,NaN,NaN,NaN,NaN,NaN,NaT


In [155]:
# Удалим дубликаты
y_obj_companies_2015.sort_values(by=["linkcut", 'id', 'normalized_name', 'funding_total_usd', 'country', 'updated_at'],
                                ascending=[True, True, True, False, True, False], inplace=True)
y_obj_companies_2015.drop_duplicates(subset=['linkcut', 'normalized_name'], keep='first', inplace=True)

In [156]:
# y_obj_companies_2015.to_pickle('y_obj_companies_2016.pkl')
# y_obj_companies_2015 = pd.read_pickle('y_obj_companies_2016.pkl')

In [157]:
# Загрузим спарсенную бд с theorg.com

theorg = pd.read_excel('result 1.xlsx')
theorg.shape

(765, 10)

In [158]:
theorg.sample()

,Company_name,City,Country,Industries,Employees,Company_website,Company_values,Org_structure,Company_description,Teams and qty
303,MagicBus,Los Angeles,United States,"Enterprise Software, Rail, Transportation",1-10,https://www.magicbus.io/,NaN,"{""p-3193650"": {""Name"": ""ARVINDS ARVI"", ""Positi...",MagicBus provides long-distance commuters a st...,"{""Leadership Team"": 3, ""CEO and Executive Team..."


In [159]:
theorg['normalized_name'] = theorg.Company_name.str.lower()

In [160]:
# Проверим, сколько дубликатов имен в y_obj_companies_2015 на текущий момент.
# Причины дублей: филиалы в разных городах, различные подразделения и т.д.
# Эти строки можно вычистить только вручную

y_obj_companies_2015[y_obj_companies_2015.normalized_name.duplicated(keep=False)].shape

(1964, 43)

In [161]:
# А всего данных в y_obj_companies_2015

y_obj_companies_2015.shape

(225674, 43)

In [316]:
# Объединим базы данных obj_companies_2015 и parse_df

t_y_obj_companies = y_obj_companies_2015.merge(theorg, on='normalized_name', how='outer')

In [317]:
# Посмотрим, насколько выросла бд
# Колонки надо будет чистить

t_y_obj_companies.shape

(226175, 53)

In [318]:
# Заполнив пустые значения из новых данных

t_y_obj_companies.name = t_y_obj_companies.name.fillna(t_y_obj_companies.Company_name)
t_y_obj_companies.city = t_y_obj_companies.city.fillna(t_y_obj_companies.City)
t_y_obj_companies.country = t_y_obj_companies.country.fillna(t_y_obj_companies.Country)
t_y_obj_companies.category_code = t_y_obj_companies.category_code.fillna(t_y_obj_companies.Industries.apply(lambda x: x if pd.isna(x) else x.split(',')[0]))
t_y_obj_companies.category_list = t_y_obj_companies.category_list.fillna(t_y_obj_companies.Industries)
t_y_obj_companies.homepage_url = t_y_obj_companies.homepage_url.fillna(t_y_obj_companies.Company_website)
t_y_obj_companies.overview = t_y_obj_companies.overview.fillna(t_y_obj_companies.Company_description)

In [319]:
# Проверим непротиворечивость полей City, Country, Company_website
# К сожалению, есть противоречивые данные, хоть и немного (112). Это надо проверять вручную : либо изменились данные, либо речь идет о нескольких филиалах
# Переименуем данные поля в _2023

t_y_obj_companies[(t_y_obj_companies.city != t_y_obj_companies.City) & t_y_obj_companies.City.notna()]

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,description,overview,tag_list,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,updated_at,linkcut,category_list,crunchbase_profile,country,founders,investors,address,state,seed_profile,y_combinator_year,Company_name,City,Country,Industries,Employees,Company_website,Company_values,Org_structure,Company_description,Teams and qty
7489,c:230550,230550.0,AirHelp,airhelp,legal,1.0,2013-01-25,NaT,airhelp.eu,http://www.getairhelp.com,theairhelper,http://s3.amazonaws.com/crunchbase_prod_assets...,6125.0,1536.0,Flight compensation,AirHelp helps air passengers get compensation ...,"legal-online-service, consumer-rights, aviatio...",NY,New York,New York City,NaT,NaT,0.0,0.0,2014-04-01,2014-04-01,1.0,4700000.0,2013-05-01,2013-10-28,2.0,3.0,2014-02-01 00:00:00,airhelp,"Legal,Transportation",http://www.crunchbase.com/organization/airhelp,United States,NaN,Y Combinator,NaN,California,http://www.seed-db.com/companies/view?companyi...,2014-02-01,AirHelp,Gdańsk,Poland,Aviation & Aerospace,500-1000,https://www.airhelp.com/,NaN,"{""p-17100"": {""Name"": ""Christian Nielsen"", ""Pos...",We’re a Y Combinator backed startup with ambit...,"{""Leadership Team"": 9, ""CEO and Executive Team..."
8143,c:53608,53608.0,Alchemy,alchemy,software,1.0,1996-01-01,NaT,alchemy.fr,http://www.alchemy.fr/spip.php?page=indexEn,AlchemyParis,http://s3.amazonaws.com/crunchbase_prod_assets...,1600.0,1199.0,Software / DAM Vendor,Alchemy is a French company specialized in the...,"photo, video, management, dam, software, open-...",NaN,Paris,Paris,NaT,NaT,0.0,0.0,NaT,NaT,0.0,0.0,2004-01-01,2012-07-24,2.0,1.0,2013-08-27 10:27:08,alchemy,Software,http://www.crunchbase.com/company/alchemy,France,NaN,NaN,NaN,NaN,NaN,NaT,Alchemy,Irvine,United States,Software,50-200,http://trustalchemy.com,NaN,"{""p-807517"": {""Name"": ""Renan Rollo"", ""Position...","Alchemy powers Banks, FinTech, and Merchants w...","{""Leadership Team"": 10, ""CEO and Executive Tea..."
10848,NaN,NaN,Amplitude,amplitude,Electronics,1.0,2001-01-01,NaT,NaN,http://amplitude-technologies.com/,NaN,NaN,NaN,NaN,NaN,Amplitude is the Digital Optimization System. ...,NaN,A8,Évry,Ã‰vry,NaT,NaT,NaN,NaN,2012-01-25,2012-01-25,1.0,39079461.0,NaT,NaT,NaN,NaN,2012-01-25 00:00:00,amplitude-2,Electronics,http://www.crunchbase.com/organization/amplitu...,France,NaN,NaN,NaN,NaN,NaN,NaT,Amplitude,San Francisco,United States,"Analytics, Information Technology, Software",500-1000,https://amplitude.com,NaN,"{""p-710228"": {""Name"": ""Hien Phan"", ""Position"":...",Amplitude is the Digital Optimization System. ...,"{""Leadership Team"": 8, ""Board of Directors"": 7..."
11141,c:32722,32722.0,Anchor,anchor,network_hosting,1.0,2000-07-20,NaT,anchor.com.au,http://www.anchor.com.au,anchorsystems,http://s3.amazonaws.com/crunchbase_prod_assets...,162.0,61.0,Australian Web Hosting,Anchor is an Australian based hosting company ...,"web-hosting, hosting, vps, dedicated-servers, ...",NaN,Sydney,Sydney,NaT,NaT,0.0,0.0,NaT,NaT,0.0,0.0,2013-03-19,2013-09-18,2.0,1.0,2009-09-30 17:37:24,anchor,Cyber Security,http://www.crunchbase.com/company/anchor,Australia,NaN,NaN,NaN,NaN,NaN,NaT,Anchor,Columbus,United States,Cyber Security,10-50,http://anchormydata.com,NaN,"{""p-4051835"": {""Name"": ""Jonathan G."", ""Positio...",Anchor is a powerfully simple way for business...,"{""Leadership Team"": 4, ""CEO and Executive Team..."
14291,NaN,NaN,Argo,argo,Data Integration,1.0,2014-04-01,NaT,NaN,https://argo.io,NaN,NaN,NaN,NaN,NaN,ARGO turns your documents into live business s...,NaN,CA,Palo Alto,SF Bay Area,NaT,NaT,NaN,NaN,2015-01-13,2015-01-13,1.0,1525000.0,NaT,NaT,NaN,NaN,2015-01-13 00:00:00,argo,"Data Integration,Data Visualization,Enterprise.

In [320]:
# Переименуем поля City, Country, Company_website в _2023
# Удалим Company_name и Company_description

t_y_obj_companies.rename(columns={'City': 'city_2023', 'Country': 'country_2023', 'Company_website': 'homepage_url_2023', 'description': 'short_description'}, inplace=True)
t_y_obj_companies.drop(columns=['Company_name', 'Company_description'], inplace=True)

In [321]:
# Заполнима пустоты в дф

# Заполним столбец domain
from urllib.parse import urlparse

def domain_extarct(url):
    if isinstance(url, str):
        parsed_url = urlparse(url)
        domain_name = parsed_url.netloc
        return domain_name.lstrip('www.')
    else:
        return url
    
t_y_obj_companies.domain = t_y_obj_companies.domain.fillna(t_y_obj_companies.homepage_url.apply(domain_extarct))

In [322]:
t_y_obj_companies.head(3)

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,overview,tag_list,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,updated_at,linkcut,category_list,crunchbase_profile,country,founders,investors,address,state,seed_profile,y_combinator_year,city_2023,country_2023,Industries,Employees,homepage_url_2023,Company_values,Org_structure,Teams and qty
0,NaN,NaN,#fame,#fame,Media,1.0,NaT,NaT,livfame.com,http://livfame.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Mumbai,Mumbai,NaT,NaT,NaN,NaN,2015-01-05,2015-01-05,1.0,10000000.0,NaT,NaT,NaN,NaN,2015-01-05,-fame,Media,http://www.crunchbase.com/organization/-fame,India,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,:Qounter,:qounter,Application Platforms,1.0,2014-09-04,NaT,qounter.com,http://www.qounter.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,Delaware City,DE - Other,NaT,NaT,NaN,NaN,2014-03-01,2014-10-14,2.0,700000.0,NaT,NaT,NaN,NaN,2014-10-14,-qounter,"Application Platforms,Real Time,Social Network...",http://www.crunchbase.com/organization/-qounter,United States,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"(THE) ONE of THEM,Inc.","(the) one of them,inc.",Apps,1.0,NaT,NaT,oneofthem.jp,http://oneofthem.jp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2014-01-30,2014-01-30,1.0,3406878.0,NaT,NaT,NaN,NaN,2014-01-30,-the-one-of-them-inc-,"Apps,Games,Mobile",http://www.crunchbase.com/organization/-the-on...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [325]:
# t_y_obj_companies.to_pickle('t_y_obj_companies.pkl')


In [324]:
# Проверим, что получилось
t_y_obj_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226175 entries, 0 to 226174
Data columns (total 51 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   196512 non-null  object        
 1   entity_id            196512 non-null  float64       
 2   name                 226173 non-null  object        
 3   normalized_name      226173 non-null  object        
 4   category_code        152500 non-null  object        
 5   status               225674 non-null  float64       
 6   founded_at           115217 non-null  datetime64[ns]
 7   closed_at            2620 non-null    datetime64[ns]
 8   domain               154653 non-null  object        
 9   homepage_url         154610 non-null  object        
 10  twitter_username     80587 non-null   object        
 11  logo_url             110103 non-null  object        
 12  logo_width           196512 non-null  float64       
 13  logo_height   

Опишем признаки:
id                     'type of object': id 
entity_id              id
name                   name of a company
normalized_name        name.lower()
category_code          category
status                 status on 2016 (0-none, 1-operating, 2-invested, 3-ipo, 4-closed)
founded_at             Date the entity was founded at
closed_at              Date the entity was closed at
domain                 URL Domain 
homepage_url           URL of entity's official site
twitter_username       official Twitter account
logo_url               Logo URL
logo_width              
logo_height             
short_description      short description. tag_list might be extracted from it
overview               long description. but might differ from short_description field
tag_list                
country_code            
state_code              
city                    
region                  
first_investment_at    Date of the first made investment (agregated crunch and y-combinator)
last_investment_at     Date of the latest made investment
investment_rounds      Number of investment rounds participated in
invested_companies     Number of companies invested in
first_funding_at       Date of the first funding round
last_funding_at        Date of the latest funding round
funding_rounds         Number of funding rounds
funding_total_usd      Total funds raised in USD (if 1 at the end - it means the precise amount is unknown, but more than it's written)
first_milestone_at     Date of the first milestone
last_milestone_at      Date of the latest milestone
milestones             Number of milestones the entity has (can be got from milestones.csv)
relationships          Number of relationships the entity has (can be got from persons and relationships.csv)
updated_at             Date the record was updated at (if nan - filled with the last event for the entity)
category_list          List of categories
crunchbase_profile     Link to page on crunch
country                Location country
founders               Founders names (can be got from persons and relationships.csv)
investors              Investors (can be got from funds and investments.csv
address                Office address
state
seed_profile           Y-combinator profile
y_combinator_year      The year of participation in Y-comb
city_2023              Left to check if there is a relocation or a new office's been opened
country_2023
homepage_url_2023
Industries
Employees              Number of emplyees
Company_values
Org_structure          Organization structure. Should be refilled with persons and relationships.csv
Teams and qty          Top teams. Should be refilled with persons and relationships.csv